# Ruin Analysis Plots

## Overview
- **What**: Deep dive into two critical risk visualizations -- the **Ruin Cliff** (retention vs. failure probability) and the **ROE-Ruin Efficient Frontier** (risk-return tradeoff with sweet-spot detection).
- **Prerequisites**: [02_executive_dashboards](02_executive_dashboards.ipynb), [../core/06_ruin_analysis](../core/06_ruin_analysis.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Practitioner]

## Topics Covered
1. Ruin Cliff -- basic, customized, and clean styles
2. Ruin Cliff with simulation data
3. ROE-Ruin Frontier -- default, linear scale, and focused views
4. Sweet-spot analysis (knee-point detection)
5. Custom color schemes and export options

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/visualization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

from ergodic_insurance.visualization import plot_ruin_cliff
from ergodic_insurance.visualization.executive_plots import (
    plot_roe_ruin_frontier,
    _find_knee_point,
)

np.random.seed(42)

%matplotlib inline
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

## 1. The Ruin Cliff -- Default View

Shows how retention levels affect ruin probability. Features include 3D gradient effects, cliff-edge detection, warning zones, and an inset for the critical transition region.

In [ ]:
fig = plot_ruin_cliff(
    title="The Ruin Cliff: Insurance Retention vs Failure Risk",
)
plt.show()

print("Zone colors:")
print("  Green:  < 2% ruin probability (safe)")
print("  Orange: 2-5% ruin probability (warning)")
print("  Red:    > 5% ruin probability (danger)")

## 2. Customized Retention Range

Adjust the retention range and company size for a medium-enterprise focus.

In [ ]:
fig = plot_ruin_cliff(
    retention_range=(5_000, 5_000_000),
    n_points=60,
    title="Ruin Cliff Analysis: Medium Enterprise Focus",
    company_size=25_000_000,
)
plt.show()

## 3. Clean Style (No 3D Effects)

Minimal version suitable for formal reports and publications.

In [ ]:
fig = plot_ruin_cliff(
    show_3d_effect=False,
    show_warnings=False,
    title="Retention vs Ruin Probability (Clean Style)",
    figsize=(12, 6),
)
plt.show()

## 4. Ruin Cliff with Simulation Data

Pass actual simulation results (e.g., from `RuinProbabilityAnalyzer`) to replace synthetic data.

In [ ]:
# Generate realistic simulation data
n_points = 75
retentions = np.logspace(4, 7, n_points)
log_ret = np.log10(retentions)
log_ret_norm = (log_ret - log_ret.min()) / (log_ret.max() - log_ret.min())

CLIFF_POSITION = 0.2
CLIFF_STEEPNESS = 20
ruin_probs = 1 / (1 + np.exp(CLIFF_STEEPNESS * (log_ret_norm - CLIFF_POSITION)))
ruin_probs = np.clip(
    ruin_probs + np.random.RandomState(42).normal(0, 0.005, n_points),
    0.001, 0.95,
)

simulation_data = {"retentions": retentions, "ruin_probs": ruin_probs}

fig = plot_ruin_cliff(
    simulation_data=simulation_data,
    title="Ruin Cliff from Simulation Results",
    company_size=50_000_000,
)
plt.show()

cliff_idx = np.argmax(np.abs(np.gradient(ruin_probs)))
print(f"Cliff edge at retention: ${retentions[cliff_idx]:,.0f}")
print(f"Ruin probability at cliff: {ruin_probs[cliff_idx]:.1%}")

## 5. ROE-Ruin Efficient Frontier -- Default View

The Pareto frontier of tradeoffs between Return on Equity and Ruin Probability across multiple company sizes, with sweet-spot markers.

In [ ]:
def generate_pareto_frontier(company_size, n_points=15):
    """Generate synthetic Pareto frontier data."""
    if company_size == 1e6:
        roe_range, ruin_range = (0.05, 0.22), (0.008, 0.10)
    elif company_size == 1e7:
        roe_range, ruin_range = (0.04, 0.16), (0.005, 0.06)
    else:
        roe_range, ruin_range = (0.03, 0.12), (0.003, 0.04)
    t = np.linspace(0, 1, n_points)
    roe = roe_range[0] + (roe_range[1] - roe_range[0]) * t
    ruin = ruin_range[1] * np.exp(-3 * t) + ruin_range[0]
    roe += np.random.normal(0, 0.003, n_points)
    ruin += np.random.normal(0, 0.001, n_points)
    roe = np.clip(roe, 0.01, 0.30)
    ruin = np.clip(ruin, 0.001, 0.15)
    return pd.DataFrame({"roe": roe, "ruin_prob": ruin, "company_size": company_size})


optimization_results = {
    1e6: generate_pareto_frontier(1e6),
    1e7: generate_pareto_frontier(1e7),
    1e8: generate_pareto_frontier(1e8),
}

fig = plot_roe_ruin_frontier(optimization_results)
plt.show()

## 6. Linear Scale View

Better for presentations where relative differences matter more than orders of magnitude.

In [ ]:
fig = plot_roe_ruin_frontier(
    optimization_results,
    title="Insurance Optimization: ROE vs. Ruin Probability",
    log_scale_y=False,
    figsize=(14, 8),
)
plt.show()

## 7. Focused Comparison

Compare only selected company sizes for a targeted analysis.

In [ ]:
fig = plot_roe_ruin_frontier(
    optimization_results,
    company_sizes=[1e6, 1e7],
    title="Small to Medium Company Comparison",
    figsize=(12, 7),
)
plt.show()

## 8. Sweet-Spot Analysis

Knee-point detection identifies where the marginal improvement in ROE no longer justifies the increase in ruin probability.

In [ ]:
print("Sweet Spot Analysis:")
print("=" * 50)

for size, df in optimization_results.items():
    df_sorted = df.sort_values("roe")
    roe_vals = df_sorted["roe"].values
    ruin_vals = df_sorted["ruin_prob"].values
    knee_idx = _find_knee_point(roe_vals, ruin_vals)
    sweet_roe = roe_vals[knee_idx]
    sweet_ruin = ruin_vals[knee_idx]
    print(f"\n${size:,.0f} Company:")
    print(f"  Sweet Spot ROE:    {sweet_roe:.1%}")
    print(f"  Sweet Spot Ruin:   {sweet_ruin:.2%}")
    print(f"  Risk-Return Ratio: {sweet_roe / sweet_ruin:.1f}")

## 9. Custom Color Scheme

Override colors for corporate branding or publication requirements.

In [ ]:
corporate_colors = ["#003366", "#0066CC", "#3399FF"]

fig = plot_roe_ruin_frontier(
    optimization_results,
    title="Insurance Portfolio Optimization",
    color_scheme=corporate_colors,
    figsize=(12, 8),
)
plt.show()

## Key Takeaways

- **Ruin Cliff**: A sharp retention threshold exists where failure risk increases dramatically. Visualize it before setting deductibles.
- **3D effects** add visual impact for executive audiences; disable them for formal publications.
- **ROE-Ruin Frontier**: The Pareto curve defines the best achievable balance between returns and safety.
- **Sweet spots** (knee points) mark the optimal insurance purchasing level for each company size.
- Both visualizations accept real simulation data or generate synthetic demonstrations.

## Next Steps

- [06_scenario_comparison](06_scenario_comparison.ipynb) -- compare multiple insurance scenarios
- [02_executive_dashboards](02_executive_dashboards.ipynb) -- full executive visualization suite
- [../core/06_ruin_analysis](../core/06_ruin_analysis.ipynb) -- ruin probability calculations